# Dying of the bees
## Correlation between the use of selected pesticides and the bee colony loss rate for each U.S. state

# Content
1. Data Collection
        1.1 Pesticide use dataset
        1.2 Bees dataset
2. Prepare the datasets
        2.1 Prepare Pesticide-Use dataset
        2.2 Prepare Bee-Colony-Loss dataset
3. Analyze for correlation
5. Conclusion

# 1. Data Collection

To the fetch the data simply run following command: "python3 read_data.py"  

In [ ]:
%run read_data.py

Directory  data  already exists


To find a correlation between the use of selected pesticides and the annual bee colony loss, I used two different sources for the datasets, which will be explained later in detail. 

### 1.1 Pesticide dataset

The first dataset - use of pesticdes - is derived from following source: https://water.usgs.gov/nawqa/pnsp/usage/maps/
The NAWQA Pesticide National Synthesis Project, which began in 1992, is a national-scale assessment of the occurrence and behavior of pesticides in streams and ground water of the United States and the potential for pesticides to adversely affect drinking-water supplies or aquatic ecosystems.

For all States except California, proprietary farm survey pesticide-use data are aggregated and reported at the multi-county *Crop Reporting District (CRD)* level. 
The USGS (U.S. Geological Survey) has estimated how various pesticides are used in agriculture in the different US states through those CRD's.

The USGS (U.S. Geological Survey) started collecting data from year 1992.
For this examination the years from 2013 until 2016 are going to be used.

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df_pesticide_use_2013 = pd.read_csv('data/pesticide_use_2013_dataset.txt', index_col=2, delimiter="\t",)
df_pesticide_use_2014 = pd.read_csv('data/pesticide_use_2014_dataset.txt', index_col=2, delimiter="\t")
df_pesticide_use_2015 = pd.read_csv('data/2015PreliminaryEstimates/EPest.county.estimates.2015.txt', index_col=2, delimiter="\t")
df_pesticide_use_2016 = pd.read_csv('data/2106PreliminaryEstimates/EPest.county.estimates.2016.txt', index_col=2, delimiter="\t")

In [ ]:
#df_pesticide_use_2013 = df_pesticide_use_2013.reset_index()
#df_pesticide_use_2013 = df_pesticide_use_2013.groupby(['STATE_FIPS_CODE']).agg({'EPEST_HIGH_KG': np.sum, 'YEAR': 'first'})

In [ ]:
number_of_datapoints_pesticides = df_pesticide_use_2013.size + df_pesticide_use_2014.size + df_pesticide_use_2015.size + df_pesticide_use_2016.size
print("number of datapoints: ",  number_of_datapoints_pesticides)

In [ ]:
df_pesticide_use_2013.head()

### 1.2 Bees dataset

The second dataset - bee colony loss - is derived from following source: https://data.world/makeovermonday/2018w18-bee-colony-loss 
The dataset is originally from https://beeinformed.org/, which is in an organisation who gathers and analyzes data about bees.

Beeinformed started collecting data from year 2013.
For this examination the years from 2013 until 2016 are going to be used.

In [ ]:
df_bee_colony_loss = pd.read_excel('data/bee_colony_loss.xlsx', index_col=2)
df_bee_colony_loss = df_bee_colony_loss.sort_values('Year')

In [ ]:
number_of_datapoints_bee_colony_loss = df_bee_colony_loss.size
print("number of datapoints: ",  number_of_datapoints_bee_colony_loss)

The organisation Beeinformed gathers information submitted by the beekeepers. The main goals of this information is to find out, 
1. how many hives were lost and
2. how healthy the bees are

In [ ]:
df_bee_colony_loss.head()

# 2. Prepare datasets

### 2.1 Pesticide datasets

The attribute to examine is the pesticide use. Herefor the USGS used two different methods, *EPest-low* and *EPest- high*(Estimation-Pesticide low and high), to estimate a range of pesticide by crop use. 
If there already exists CRD's including pesticide use by crop, then the USGS applied these survyed rate.
If use of a pesticide on a crop was not reported in a surveyed CRD, EPest-low reports zero use in the CRD for that pesticide-by-crop combination. EPest-high, however, treats the unreported use for that pesticide-by-crop combination in the CRD as unsurveyed, and pesticide-by-crop use rates from neighboring CRDs and, in some cases, CRDs within the same USDA Farm Resource Region are used to calculate the pesticide-by-crop EPest-high rate for the CRD.

The dataset contains following attributes:
- COMPOUND: The kind of pesticide 
- YEAR: Year of the CRD
- STATE_FIPS_CODE: code which uniquely identified states
- COUNTY_FIPS_CODE: code which uniquely identified counties
- EPEST_LOW_KG: low estimation of pesticide use
- EPEST_HIGH_KG: high estimation of pesticide use

For this examination we only compare the values of seven selected Neonicotinoids, as studies have shown that Neonicotinoids are the most harmful pesticides for bees (https://www.compoundchem.com/2015/04/14/neonicotinoids/, 
https://www.hsph.harvard.edu/news/press-releases/study-strengthens-link-between-neonicotinoids-and-collapse-of-honey-bee-colonies/)

Following Neonicotinoids are going to be used for the examination:
- Acetamiprid
- Clothianidin
- Dinotefuran
- Imidacloprid
- Thiacloprid
- Thiamethoxam

In [ ]:
#imports and styling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-muted')

In [ ]:
# year and county code is not needed
df_pesticide_use_2013 = df_pesticide_use_2013.drop(columns=['YEAR','COUNTY_FIPS_CODE'])
df_pesticide_use_2014 = df_pesticide_use_2014.drop(columns=['YEAR','COUNTY_FIPS_CODE'])
df_pesticide_use_2015 = df_pesticide_use_2015.drop(columns=['YEAR','COUNTY_FIPS_CODE'])
df_pesticide_use_2016 = df_pesticide_use_2016.drop(columns=['YEAR','COUNTY_FIPS_CODE'])

In [ ]:
# sort by states
df_pesticide_use_2013 = df_pesticide_use_2013.sort_index()
df_pesticide_use_2014 = df_pesticide_use_2014.sort_index()
df_pesticide_use_2015 = df_pesticide_use_2015.sort_index()
df_pesticide_use_2016 = df_pesticide_use_2016.sort_index()

EPEST_LOW_KG and EPEST_HIGH_KG are estimations in Kilos on the amount of pesticides used for each county:

In [ ]:
print("Datatypes of Estimations: ", df_pesticide_use_2013.EPEST_LOW_KG.dtype)

In [ ]:
# only get the important pesticides
important_pesticides = ['CLOTHIANIDIN', 'DINOTEFURAN', 'IMIDACLOPRID', 'THIAMETHOXAM', 'ACETAMIPRID']
def important_pesticides_only(pesticide_dataframe):
    important_pesticides_dataframes = []
    for i in range(0, len(important_pesticides)):
        important_pesticides_dataframes.append(pesticide_dataframe[pesticide_dataframe.COMPOUND == important_pesticides[i]])
    return important_pesticides_dataframes

In [ ]:
# Combine the dataframes with the correct pesticides for each year
df_pesticide_use_2013_with_correct_pesticides = pd.concat(important_pesticides_only(df_pesticide_use_2013))
df_pesticide_use_2014_with_correct_pesticides = pd.concat(important_pesticides_only(df_pesticide_use_2014))
df_pesticide_use_2015_with_correct_pesticides = pd.concat(important_pesticides_only(df_pesticide_use_2015))
df_pesticide_use_2016_with_correct_pesticides = pd.concat(important_pesticides_only(df_pesticide_use_2016))

Now the dataframes contains only the pesticides we're going to need. At the moment, for each state there are multiple entries, due to the different counties. 
As we just want to compare the different states, we combine all values within a state.
To analyze the use of the selected pesticides for each state we get sum of the estimations.

In [ ]:
list_pesticide_use_2013_to_2016 = [df_pesticide_use_2013_with_correct_pesticides, 
                                   df_pesticide_use_2014_with_correct_pesticides, 
                                   df_pesticide_use_2015_with_correct_pesticides, 
                                   df_pesticide_use_2016_with_correct_pesticides]

In [ ]:
df_pesticide_use_2013_to_2016_with_correct_pesticides = pd.concat(list_pesticide_use_2013_to_2016)

In [ ]:
# group all state codes and get the sum of EPEST_LOW_KG and EPEST_HIGH_KG for each Pesticide
df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped = round(df_pesticide_use_2013_to_2016_with_correct_pesticides.groupby(['STATE_FIPS_CODE']).agg('sum') ,0)

In [ ]:
df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped.head()

### 2.2 Bee Colony Loss dataset

Beeinformed gathers information of the beekeepers and compare the amount of colonies to the previous year, so that they can calculate a percentage loss.

- Year: Year of examination
- Season: Season of examination
- State: All states of the US 
- Total Annual Loss: Loss of colonies within this year, in percentage
- Beekeepers: Amount of beekeepers who has submitted information
- Colonies: Amount of colonies

In [ ]:
# remove white space in columns
df_bee_colony_loss.columns = df_bee_colony_loss.columns.str.replace(' ', '')
df_bee_colony_loss.columns = df_bee_colony_loss.columns.str.lstrip()

As we just need the annual loss of the colonies, we drop every other column except the year

In [ ]:
# remove not needed columns
df_bee_colony_loss = df_bee_colony_loss.drop(columns=['Season', 'BeekeepersExclusivetoState', 'Beekeepers', 'Colonies', 'ColoniesExclusivetoState'])

In [ ]:
# sort by states
df_bee_colony_loss_sorted = df_bee_colony_loss.sort_index()

In [ ]:
# only get the years 2013-2016
df_bee_colony_loss_sorted_2013_to_2016_ = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year != '2010/11']
df_bee_colony_loss_sorted_2013_to_2016_ = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year != '2011/12']

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016_.head()

Now I have the dataframe with the annual loss of the bee colonies for the years of 2013-2016.
To compare the data with the use of pesticides we need to group them in the different years.

In [ ]:
# create new dataframes for each year
df_bee_colony_loss_sorted_2013 = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year == '2013/14']
df_bee_colony_loss_sorted_2014 = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year == '2014/15']
df_bee_colony_loss_sorted_2015 = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year == '2015/16']
df_bee_colony_loss_sorted_2016 = df_bee_colony_loss_sorted[df_bee_colony_loss_sorted.Year == '2016/17']

Some states, e.g Alaska, dont't have information about their bee colony loss.
Also, the annual loss is of datatype float and is presented as a percentage. For a more understandable visualization, we round the percentage value to zero decimalpoints.

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016 = df_bee_colony_loss_sorted_2013_to_2016_.dropna()

In [ ]:
#df_bee_colony_loss_sorted_2013_to_2016 = df_bee_colony_loss_sorted_2013_to_2016[df_bee_colony_loss_sorted_2013_to_2016.select_dtypes(include=['number']).columns]
state_count = df_bee_colony_loss_sorted_2013_to_2016.loc[df_bee_colony_loss_sorted_2013_to_2016.index == 'Arkansas'].index.value_counts()
print(state_count)

In [ ]:
#each state appears 6 times
df_bee_colony_loss_sorted_2013_to_2016 = round((df_bee_colony_loss_sorted_2013_to_2016.groupby(df_bee_colony_loss_sorted_2013_to_2016.index).agg('sum') / 6) * 100,0)
df_bee_colony_loss_sorted_2013_to_2016.head()

# 3. Analyze correlation

Comparison between two dataframes for the years 2013-2016 with following variables:
- Categorial Variables: States and State_FIPS_Code
- Discrete Variables: Pesticide-use Estimation and Total Annual Loss

In [ ]:
df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped.head()

In [ ]:
df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped.count()

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016.head()

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016.count()

Hereby i found out that the amount of rows differ by 3. To analyze which states are missing i put both dataframes side by side and compared them by FIPS-Code from: https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code

In [ ]:
def dataframes_comparisor(): 
    df1_styler = df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped.style.set_table_attributes("style='display:inline'").set_caption('pesticides')
    df2_styler = df_bee_colony_loss_sorted_2013_to_2016.style.set_table_attributes("style='display:inline'").set_caption('colony loss')
    from IPython.display import display_html
    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

In [ ]:
dataframes_comparisor()

Following states are missing within the pesticide dataset: 
- District of Columbia
- Hawaii
- MultiStateOperation

These states will be removed within the bee dataset

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016 = df_bee_colony_loss_sorted_2013_to_2016.drop(index=['District of Columbia', 'Hawaii', 'MultiStateOperation'])
df_bee_colony_loss_sorted_2013_to_2016.count()

Now i can merge the two datasets, as they have the same states for each row.

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016 = df_bee_colony_loss_sorted_2013_to_2016.reset_index(drop=True)
df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped = df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped.reset_index(drop=True)
df_bees_and_pesticides_2013_to_2016 = df_bee_colony_loss_sorted_2013_to_2016.join(df_pesticide_use_2013_to_2016_with_correct_pesticides_grouped)

In [ ]:
def jointplot_selected_pesticides():
        ax = sns.jointplot("TotalAnnualLoss", "EPEST_HIGH_KG", data=df_bees_and_pesticides_2013_to_2016, kind="reg", height=7)
        plt.title('Correlation between colony annual loss and the amount of selected pesticide use estimation from 2013 to 2016', y=1.25)

In [ ]:
jointplot_selected_pesticides()

We can't see any clear correlation between the amount of pesticide use and the annual loss.
Maybe the selected pesticides aren't that influential. Lets check if there is a correlation when all pesticides are involved. 

In [ ]:
list_pesticide_use_2013_to_2016_all_pesticides = [df_pesticide_use_2013, df_pesticide_use_2014, df_pesticide_use_2015,
                                         df_pesticide_use_2016]
df_pesticide_use_2013_to_2016_all_pesticides = pd.concat(list_pesticide_use_2013_to_2016_all_pesticides)
df_pesticide_use_2013_to_2016_all_pesticides_grouped = df_pesticide_use_2013_to_2016_all_pesticides.groupby(['STATE_FIPS_CODE']).agg('sum')

In [ ]:
df_bees_and_pesticides_2013_to_2016_all_pesticides = df_bee_colony_loss_sorted_2013_to_2016.join(df_pesticide_use_2013_to_2016_all_pesticides_grouped)
df_bees_and_pesticides_2013_to_2016_all_pesticides = df_bees_and_pesticides_2013_to_2016_all_pesticides.dropna()

In [ ]:
def jointplot_all_pesticides():
        ax = sns.jointplot("TotalAnnualLoss", "EPEST_HIGH_KG", 
                           data=df_bees_and_pesticides_2013_to_2016_all_pesticides, 
                           kind="reg", height=7)
        plt.title('Correlation between colony annual loss and the amount of all pesticide use estimation from 2013 to 2016', y=1.25)

In [ ]:
jointplot_all_pesticides()

The regression line tends to be steeper than the regression line of the selected pesticides. However it still doesn't show that the pesticides have a big impact on the annual loss.

In [ ]:
def pesticide_kernel_density_estimation(): 
        f, ax = plt.subplots(figsize=(8, 8))
        ax.set_aspect("equal")
        # Draw the two density plots
        ax = sns.kdeplot(df_bees_and_pesticides_2013_to_2016_all_pesticides.TotalAnnualLoss, 
                         df_bees_and_pesticides_2013_to_2016_all_pesticides.EPEST_HIGH_KG/10000000,
                         cmap="Reds", shade=False, shade_lowest=True)
        ax = sns.kdeplot(df_bees_and_pesticides_2013_to_2016.TotalAnnualLoss, 
                         df_bees_and_pesticides_2013_to_2016.EPEST_HIGH_KG/100000,
                         cmap="Blues", shade=False, shade_lowest=True)
        red = sns.color_palette("Reds", 8)[-1]
        blue = sns.color_palette("Blues", 8)[-1]
        ax.text(15, 20, "All pesticides", size=16, color=blue)
        ax.text(15, 18, "Selected pesticides", size=16, color=red)
        ax.set_title('Difference between selected- and all pesticide in relation to the annual loss')

In [ ]:
pesticide_kernel_density_estimation()

The difference in the distribution between all pesticides and the selected pesticides is similar. 

In [ ]:
df_bees_and_pesticides_2013_to_2016_all_pesticides = df_bees_and_pesticides_2013_to_2016_all_pesticides.reset_index()
df_bees_and_pesticides_2013_to_2016_all_pesticides = df_bees_and_pesticides_2013_to_2016_all_pesticides.drop(df_bees_and_pesticides_2013_to_2016_all_pesticides.columns[2], axis=1)
df_bees_and_pesticides_2013_to_2016_all_pesticides = df_bees_and_pesticides_2013_to_2016_all_pesticides.sort_values('TotalAnnualLoss')

In [ ]:
def states_pairgrid():
        sns.set(style="whitegrid")

        g = sns.PairGrid(df_bees_and_pesticides_2013_to_2016_all_pesticides,
                         x_vars=df_bees_and_pesticides_2013_to_2016_all_pesticides.columns[1:], y_vars=["index"],
                         height=8, aspect=.4)

        g.map(sns.stripplot, size=10, orient="h",
              palette="ch:s=1,r=-.1,h=1_r", linewidth=1, edgecolor="w")

        plt.subplots_adjust(top=0.9)
        g.fig.suptitle('Correlation between the annual loss and the pesticide use estimation for each state')

        titles = ['Annual Loss', 'Pesticide Use Estimation']

        for ax, title in zip(g.axes.flat, titles):

            # Set a different title for each axes
            ax.set(title=title)

            # Make the grid horizontal instead of vertical
            ax.xaxis.grid(False)
            ax.yaxis.grid(True)

        sns.despine(left=True, bottom=True)

In [ ]:
states_pairgrid()

As seen for each line (state), there is no correlation between the amount of pesticide use and the annual loss. 
It would have been expected that the dots move equally along the line.

Let's see how the total amount of pesticide use affected the different years in comparison to the annual loss

In [ ]:
df_bee_colony_loss_sorted_2013_to_2016_ = df_bee_colony_loss_sorted_2013_to_2016_.drop(index=['District of Columbia', 'Hawaii', 'MultiStateOperation'])

In [ ]:
df_bee_colony_loss_over_years = df_bee_colony_loss_sorted_2013_to_2016_.dropna()

In [ ]:
df_bee_colony_loss_over_years.columns = df_bee_colony_loss_over_years.columns.str.lstrip()

In [ ]:
#sum divided by 48 states
df_bee_colony_loss_over_years = round((df_bee_colony_loss_over_years.groupby(df_bee_colony_loss_over_years.Year).agg('sum') / 48) * 100,0)
df_bee_colony_loss_over_years = df_bee_colony_loss_over_years.drop(df_bee_colony_loss_over_years.index[[0,3]])

In [ ]:
df_bee_colony_loss_over_years['EPEST_HIGH_KG'] = [round(df_pesticide_use_2013.EPEST_HIGH_KG.mean()), round(df_pesticide_use_2014.EPEST_HIGH_KG.mean()),
                          round(df_pesticide_use_2015.EPEST_HIGH_KG.mean()), round(df_pesticide_use_2016.EPEST_HIGH_KG.mean())]

In [ ]:
def boxplot_yearly_annualloss():
        fig, ax = plt.subplots(figsize=(10, 6))
        _ = ax.set_title('Yearly Annual Loss in relation to pesticide use')
        sns.barplot(x='EPEST_HIGH_KG', y='TotalAnnualLoss', data=df_bee_colony_loss_over_years)
        _ = plt.xlabel('Average pesticide estimation per year')

In [ ]:
boxplot_yearly_annualloss()

Expected would be that the last column (the year 2016) has the highest annual loss, because of the highest pesticide use estimation. This is not the case. Also the other year doesn't show a correlation.

# 4. Conclusion

The neonicotinoids are not as influential as first thought. However, all pesticides together also don't seem to have a big impact on the annual bee colony loss as well. Many factors such as pathogens, parasites, weather conditions, habitat loss, poor nutrition, and agriculture and apiary practice impact the health of bee colonies today. It would take a lot of data and time to consider every factor and to gain insight on which is the most harmful one. 